# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import numpy as np

In [2]:
data_0 = pd.read_csv('/datasets/geo_data_0.csv')
data_1 = pd.read_csv('/datasets/geo_data_1.csv')
data_2 = pd.read_csv('/datasets/geo_data_2.csv')

In [3]:
def information(data):
    print('    Первые 5 строк данных')
    print(data.head())
    print('\n')
    print('    Информация о таблице с данными')
    print(data.info())
    print('\n')
    print('    Наличие пропусков:')
    print(data.isna().sum())
    print('\n')
    print(f'Наличие дубликатов: {data_0.duplicated().sum()}')

### Данные о геологоразведки первого региона

In [4]:
information(data_0)

    Первые 5 строк данных
      id        f0        f1        f2     product
0  txEyH  0.705745 -0.497823  1.221170  105.280062
1  2acmU  1.334711 -0.340164  4.365080   73.037750
2  409Wp  1.022732  0.151990  1.419926   85.265647
3  iJLyR -0.032172  0.139033  2.978566  168.620776
4  Xdl7t  1.988431  0.155413  4.751769  154.036647


    Информация о таблице с данными
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None


    Наличие пропусков:
id         0
f0         0
f1         0
f2         0
product    0
dtype: int64


Наличие дубликатов: 0


### Данные о геологоразведки второго региона

In [5]:
information(data_1)

    Первые 5 строк данных
      id         f0         f1        f2     product
0  kBEdx -15.001348  -8.276000 -0.005876    3.179103
1  62mP7  14.272088  -3.475083  0.999183   26.953261
2  vyE1P   6.263187  -5.948386  5.001160  134.766305
3  KcrkZ -13.081196 -11.506057  4.999415  137.945408
4  AHL4O  12.702195  -8.147433  5.004363  134.766305


    Информация о таблице с данными
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None


    Наличие пропусков:
id         0
f0         0
f1         0
f2         0
product    0
dtype: int64


Наличие дубликатов: 0


### Данные о геологоразведки третьего региона

In [6]:
information(data_2)

    Первые 5 строк данных
      id        f0        f1        f2     product
0  fwXo0 -1.146987  0.963328 -0.828965   27.758673
1  WJtFt  0.262778  0.269839 -2.530187   56.069697
2  ovLUW  0.194587  0.289035 -5.586433   62.871910
3  q6cA6  2.236060 -0.553760  0.930038  114.572842
4  WPMUX -0.515993  1.716266  5.899011  149.600746


    Информация о таблице с данными
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None


    Наличие пропусков:
id         0
f0         0
f1         0
f2         0
product    0
dtype: int64


Наличие дубликатов: 0


Были загружены данные всех трех регионов. Пропусков, дубликатов, некорректных наименований и не соответствующих типо данных не было обнаружено. Данные готовы для анализа.

## Обучение и проверка модели

Каждый датасет разобьем на обучающую и валидационную выборки в соотношении 75:25. Для обучения будет использована модель LinearRegression. Так как в данных присутствуют количественные признаки, проведем стандартизацию данных. После этого по всем данным на валидационной выборке сделаем обучение. В результате найдем предсказанного сырья и RMSE модели.

In [7]:
def model_training(data):
    features = data.drop(["product","id"], axis=1)
    target = data["product"]
    features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size=0.25, random_state=1)

    scaler = StandardScaler().fit(features_train)

    features_train = scaler.fit_transform(features_train)
    features_valid = scaler.transform(features_valid)

    model = LinearRegression()
    model.fit(features_train, target_train)
    pred_valid = model.predict(features_valid)

    pred_valid_mean = pred_valid.mean()
    rmse = mean_squared_error(pred_valid, target_valid) ** 0.5
    
    return target_valid, pred_valid, pred_valid_mean, rmse

### Первый регион

In [8]:
target_valid_0, pred_valid_0, pred_valid_mean_0, rmse_0 = model_training(data_0)

### Второй регион

In [9]:
target_valid_1, pred_valid_1, pred_valid_mean_1, rmse_1 = model_training(data_1)

### Третий регион

In [10]:
target_valid_2, pred_valid_2, pred_valid_mean_2, rmse_2 = model_training(data_2)

### Вывод

In [11]:
print('Первый регион')
print(f'Средний запас предсказанного сырья: {pred_valid_mean_0}')
print(f'RMSE модели: {rmse_0}')
print('\n')
print('Второй регион')
print(f'Средний запас предсказанного сырья: {pred_valid_mean_1}')
print(f'RMSE модели: {rmse_1}')
print('\n')
print('Третий регион')
print(f'Средний запас предсказанного сырья: {pred_valid_mean_2}')
print(f'RMSE модели: {rmse_2}')

Первый регион
Средний запас предсказанного сырья: 92.49262459838863
RMSE модели: 37.74258669996437


Второй регион
Средний запас предсказанного сырья: 69.12040524285558
RMSE модели: 0.8943375629130574


Третий регион
Средний запас предсказанного сырья: 94.9568304858529
RMSE модели: 39.86671127773423


Рассмотрим показатели у всех трех регионов. Предсказанный средний запас сырья у первого и третьего региона почти на одном уровне. Но показатель RMSE у этих регионов слишком велик. Особенно выделяется третий регион: у него самый большой показатель запасы сырья, а также самое большое значение среднеквадратичной ошибки.


Что нельзя сказать про второй регион. Среднее расстояние между прогнозируемыми значениями из модели и фактическими значениями в наборе данных меньше 1. Это говорит о том, что в данном случае модель лучше всего можем предсказать результат.Но при этом средний запас сырья на 20 - 25 тыс. баррелей меньше по сравнению с другими регионами.

## Подготовка к расчёту прибыли

При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.
Бюджет на разработку скважин в регионе — 10 млрд рублей.
При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.

Нас интересует оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирем регион с наибольшей средней прибылью.

Для этого подготовим все данные для расчета прибыли и рисков и проведем вычисления.

In [12]:
BUDGET = 10000000000
REVENUE_UNIT_PRODUCTION = 450000
NUMBER_DEVELOPMENT_POINTS = 200
NUMBER_POINTS= 500

Чтобы возможно было оценить риски, необходимо посчитать достаточный объём сырья для безубыточной разработки новой скважины. Для этого определим, сколько тысяч баррелей рассчитано на регион и затем рассчитаем средний показатель для 200 точек добычи.

In [13]:
value = BUDGET / REVENUE_UNIT_PRODUCTION
average = value / NUMBER_DEVELOPMENT_POINTS
print('Для безубыточной разработки новой скважины необходимо (в тыс. баррелях) :')
print('Объем добываемого сырья в регионе: ', value)
print('Средний объем добываемого сырья с одной точки добычи:', average)

Для безубыточной разработки новой скважины необходимо (в тыс. баррелях) :
Объем добываемого сырья в регионе:  22222.222222222223
Средний объем добываемого сырья с одной точки добычи: 111.11111111111111


## Расчёт прибыли и рисков 

In [14]:
'''
Функция для расчёта прибыли по предсказаниям модели
'''
def profit_calculation(target, pred):
    pred = pred.sort_values(ascending=False)
    target = target[pred.index][:NUMBER_DEVELOPMENT_POINTS]
    return target.sum() * REVENUE_UNIT_PRODUCTION - BUDGET

In [15]:
'''
Функция для нахождения распределение прибыли, доверительного интервала с помощью техники Bootstrap с 1000 выборок
'''
def profit_distribution(target, pred):
    state = np.random.RandomState(12345)
    bootstrap_samples = 1000
    values = []
    target = pd.Series(target).reset_index(drop=True)
    pred = pd.Series(pred)
    for i in range(bootstrap_samples):
        subsample = target.sample(NUMBER_POINTS, replace=True, random_state=state)
        pred_s = pred[subsample.index]
        values.append(profit_calculation(subsample, pred_s))

    values = pd.Series(values)
    lower = values.quantile(0.025)
    upper = values.quantile(0.975)
    risks = (values < 0).mean()* 100
    return values, lower, upper, risks

In [16]:
profit_0, lower_0, upper_0, risks_0 = profit_distribution(target_valid_0, pred_valid_0)
profit_1, lower_1, upper_1, risks_1 = profit_distribution(target_valid_1, pred_valid_1)
profit_2, lower_2, upper_2, risks_2 = profit_distribution(target_valid_2, pred_valid_2)

In [17]:
print('Данные по регионам')
print('Первый регион')
print('Средняя прибыль:', profit_0.mean())
print('95%-й доверительный интервал:', lower_0, upper_0)
print(upper_0 - lower_0)
print('Риск убытков в процентах:', risks_0)
print('\n')
print('Второй регион')
print('Средняя прибыль:', profit_1.mean())
print('95%-й доверительный интервал:', lower_1, upper_1)
print(upper_1 - lower_1)
print('Риск убытков в процентах:', risks_1)
print('\n')
print('Третий регион')
print('Средняя прибыль:', profit_2.mean())
print('95%-й доверительный интервал:', lower_2, upper_2)
print(upper_2 - lower_2)
print('Риск убытков в процентах:', risks_2)

Данные по регионам
Первый регион
Средняя прибыль: 458100401.7997511
95%-й доверительный интервал: -91938453.32589239 1015824888.6825125
1107763342.008405
Риск убытков в процентах: 4.6


Второй регион
Средняя прибыль: 533609292.92167586
95%-й доверительный интервал: 98001407.23450962 966461912.599809
868460505.3652995
Риск убытков в процентах: 0.1


Третий регион
Средняя прибыль: 426348386.1666254
95%-й доверительный интервал: -116622627.7963096 980501749.734359
1097124377.5306687
Риск убытков в процентах: 6.1


## Заключение 

Была поставлена задача по выбору одного региона из трех представленных для бурения новой скважины. 

Для анализа данные были разбиты на обучающую и валидационную выборки в соотношении 75:25. Для обучения была выбрана модель линейная регрессия. По результатам обучения наибольший показатель предсказанного среднего запаса сырья был у третьего региона (94.9568304858529). Это самое близкое значение к рассчитанному достаточному объёму сырья для безубыточной разработки новой скважины (111). При этом лучший показатель RMSE был у второго региона.

При расчете прибыли для каждого региона было выявлено, что в во втором регионе наибольший показатель по значениям средней прибыли (533609292.92167586), в третьем регионе -  95%-й доверительного интервала (1097124377.5306687).

Также был посчитан риск убытков. Наилучший показатель был у второго региона - 0.1 %, далее идет первый регион - 4.6 % и третий регион - 6.1 %

Можно сделать вывод, что наиболее выгодным регионом для бурения новой скважины является второй регион.